In [7]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE
import pickle
from sklearn.metrics import roc_auc_score
from lightgbm.sklearn import LGBMClassifier



In [8]:
data_train = pd.read_csv("cleaned_2013_14", low_memory = False);
data_test = pd.read_csv("cleaned_2015", low_memory = False);

In [9]:
data_test = data_test.dropna()
data_test.columns == data_train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [10]:
def split_data(df, cols):
    x = df.drop(cols, axis = 1)
    y = df.paid
    return x, y

cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term_adj']
x_train_initial, y_train_initial = split_data(data_train, cols_to_drop_training)
x_test, y_test = split_data(data_test, cols_to_drop_training)

In [11]:
sm = SMOTE(random_state=1, ratio = 1.0)
x_train, y_train = sm.fit_sample(x_train_initial, y_train_initial)

ValueError: could not convert string to float: '937xx'

In [ ]:
x_train.shape, x_test.shape

In [ ]:
# Use Pipeline instead of make_pipeline to do grid search

pipe = Pipeline([('POLY', PolynomialFeatures(degree=2, include_bias=False)),
            ('LGBM',LinearDiscriminantAnalysis(n_estimators = 150, n_jobs = 3))])

# # Instantiate GS
# gs = GridSearchCV(pipe,param_grid, verbose=10, cv=4)
lgbm = pipe.fit(x_train, y_train)

In [ ]:
# #create a function to take advantage of sklearn make_pipeline
# #pipeline adds second order terms and interaction terms to X_train and then fits model

# def rf_model(x, y, d):
#     model = make_pipeline(
#             PolynomialFeatures(degree=2, include_bias=False),
#             RandomForestClassifier(max_features = 100, max_depth = d, n_estimators = 100)
#             )
#     model.fit(x, y)
#     return model

# rf_complex = rf_model(x_train, y_train, 10)